In [15]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoConfig
import re
from peft import PeftModel

In [16]:
class DebertaRegressor(nn.Module):
    def __init__(self, model_name="microsoft/deberta-v3-base"):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.deberta = AutoModel.from_pretrained(model_name, config=self.config)
        self.regressor = nn.Linear(self.config.hidden_size, 1)
        self.base_model = self.deberta

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
        return (token_embeddings * input_mask_expanded).sum(1) / input_mask_expanded.sum(1)

    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        pooled_output = self.mean_pooling(outputs, attention_mask)
        logits = self.regressor(pooled_output).squeeze(-1)
        return torch.sigmoid(logits)

class DebertaRegressorLarge(nn.Module):
    def __init__(self, model_name="microsoft/deberta-v3-large"):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.deberta = AutoModel.from_pretrained(model_name, config=self.config)
        self.regressor = nn.Linear(self.config.hidden_size, 1)
        self.base_model = self.deberta

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
        return (token_embeddings * input_mask_expanded).sum(1) / input_mask_expanded.sum(1)

    def forward(self, input_ids=None, attention_mask=None, **kwargs):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask, **kwargs)
        pooled_output = self.mean_pooling(outputs, attention_mask)
        logits = self.regressor(pooled_output).squeeze(-1)
        return torch.sigmoid(logits)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<[^>]+>', ' ', text)              
    text = re.sub(r'https?://\S+|www\.\S+', '', text) 
    text = re.sub(r"[^a-z0-9.,!?'\s]", ' ', text)      
    text = re.sub(r'\s+', ' ', text).strip()           
    return text

def predict_feedback(model, tokenizer, text, device, max_length=512):
    model.eval()
    model.to(device)
    text_cleaned = clean_text(text)

    tokens = tokenizer(
        text_cleaned,
        return_tensors="pt",
        max_length=max_length,
        padding="max_length",
        truncation=True
    )

    input_ids = tokens["input_ids"].to(device)
    attention_mask = tokens["attention_mask"].to(device)

    with torch.no_grad():
        pred = model(input_ids=input_ids, attention_mask=attention_mask)

    predicted_score = pred.item()

    
    if predicted_score >= 0.7:
        overall = "This essay is strong overall, showing excellent organization, development of ideas, and clarity in expression."
        strengths = "Ideas are well-structured, arguments are persuasive, and language use is mostly mature and accurate."
        improvements = "Polish minor grammar issues and ensure transitions are smooth to reach perfection."

    elif predicted_score >= 0.5:
        overall = "This essay is fairly effective, with generally clear ideas and good effort in structure."
        strengths = "The essay presents organized thoughts and some persuasive arguments."
        improvements = "Work on refining grammar and strengthening transitions between points."

    elif predicted_score >= 0.3:
        overall = "This essay has potential but needs improvement in structure, clarity, and development."
        strengths = "There are some clear ideas, and an effort to support them is noticeable."
        improvements = "Improve sentence flow, avoid repetition, and develop arguments more fully."

    else:
        overall = "This essay is underdeveloped and difficult to follow, with significant issues in structure and language."
        strengths = "The attempt to engage with the topic is visible."
        improvements = "Focus on organizing your thoughts clearly, using proper grammar, and fully developing your main ideas."

    return {
        "predicted_score": predicted_score,
        "overall_evaluation": overall,
        "strengths": strengths,
        "areas_for_improvement": improvements
    }



In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer_base = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
tokenizer_large = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")
base_model = DebertaRegressor().to(device)
base_model_2 = DebertaRegressorLarge().to(device)
base_model_3 = DebertaRegressorLarge().to(device)
base_model_4 = DebertaRegressorLarge().to(device)
model = PeftModel.from_pretrained(base_model, "D:/nlu/lora_deberta_adapter_final").to(device)
model_2 = PeftModel.from_pretrained(base_model_2, "D:/nlu/lora_deberta_adapter_final_2").to(device)
model_3 = PeftModel.from_pretrained(base_model_3, "D:/nlu/lora_deberta_adapter_final_3").to(device)
model.eval()
model_2.eval()
model_3.eval()

c:\Users\Mohamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): DebertaRegressorLarge(
      (deberta): DebertaV2Model(
        (embeddings): DebertaV2Embeddings(
          (word_embeddings): Embedding(128100, 1024, padding_idx=0)
          (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
          (dropout): StableDropout()
        )
        (encoder): DebertaV2Encoder(
          (layer): ModuleList(
            (0-23): 24 x DebertaV2Layer(
              (attention): DebertaV2Attention(
                (self): DisentangledSelfAttention(
                  (query_proj): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
            

In [ ]:
# Perfect essay
essay = """
In today’s increasingly complex economic landscape, financial literacy is not a luxury—it is a necessity. Despite this, many high school graduates are unequipped to handle basic financial tasks such as budgeting, saving, or understanding credit. To bridge this critical gap, financial literacy should be a mandatory component of every high school curriculum. Doing so will empower students to make informed decisions, avoid debt traps, and become financially responsible adults.

First, requiring financial literacy in high school equips students with essential life skills. According to a 2023 survey by the National Financial Educators Council, 64% of young adults felt that their lack of financial knowledge had cost them money. Teaching students how to create a budget, understand interest rates, and manage debt will not only benefit them personally but will also have long-term societal impacts by reducing poverty and reliance on public assistance.

Second, financial literacy promotes responsible consumer behavior. In an age where credit cards, student loans, and online shopping are readily accessible, young people are more vulnerable than ever to financial missteps. A well-designed course can teach students about compound interest, credit scores, and the dangers of predatory lending. With this knowledge, they are more likely to make informed choices and avoid high-risk financial behavior.

Finally, making financial education mandatory can help close the wealth gap. Studies show that students from lower-income families are less likely to receive financial education at home. By integrating it into public education, we create a more level playing field and offer every student—regardless of background—the opportunity to succeed financially.

In conclusion, requiring financial literacy in high school is a powerful and practical step toward building a more financially stable and informed society. It prepares students for real-world challenges, fosters responsible habits, and promotes equity. In a world driven by money, understanding how to manage it is not just important—it is essential.
"""
print("Deberta Base weighted")
result = predict_feedback(model, tokenizer_base, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])

print("Deberta Large weighted")
result = predict_feedback(model_2, tokenizer_large, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])

print("Deberta Large Balanced")
result = predict_feedback(model_3, tokenizer_large, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])

print("Base Deberta model")
result = predict_feedback(base_model_4, tokenizer_large, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])


Deberta Base weighted
Predicted Score: 0.6751041412353516
Overall Evaluation: This essay is fairly effective, with generally clear ideas and good effort in structure.
Strengths: The essay presents organized thoughts and some persuasive arguments.
Areas for Improvement: Work on refining grammar and strengthening transitions between points.
Deberta Large weighted
Predicted Score: 0.7091741561889648
Overall Evaluation: This essay is strong overall, showing excellent organization, development of ideas, and clarity in expression.
Strengths: Ideas are well-structured, arguments are persuasive, and language use is mostly mature and accurate.
Areas for Improvement: Polish minor grammar issues and ensure transitions are smooth to reach perfection.
Deberta Large Balanced
Predicted Score: 0.8171554207801819
Overall Evaluation: This essay is strong overall, showing excellent organization, development of ideas, and clarity in expression.
Strengths: Ideas are well-structured, arguments are persuasiv

In [ ]:
# Bad essay
essay = """The sky is blue and water is wet. I have a pencil that I use to write sometimes when I think about things like clouds or maybe animals. This is writing. I like pizza. Sometimes people say stuff about important things but I don’t know what they mean. Writing is hard when the sun is shining and it’s hot. That’s all I know today."""

print("Deberta Base weighted")
result = predict_feedback(model, tokenizer_base, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])

print("Deberta Large weighted")
result = predict_feedback(model_2, tokenizer_large, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])

print("Deberta Large Balanced")
result = predict_feedback(model_3, tokenizer_large, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])

print("Base Deberta model")
result = predict_feedback(base_model_4, tokenizer_large, essay, device)
print("Predicted Score:", result['predicted_score'])
print("Overall Evaluation:", result['overall_evaluation'])
print("Strengths:", result['strengths'])
print("Areas for Improvement:", result['areas_for_improvement'])



Deberta Base weighted
Predicted Score: 0.49404025077819824
Overall Evaluation: This essay has potential but needs improvement in structure, clarity, and development.
Strengths: There are some clear ideas, and an effort to support them is noticeable.
Areas for Improvement: Improve sentence flow, avoid repetition, and develop arguments more fully.
Deberta Large weighted
Predicted Score: 0.011292065493762493
Overall Evaluation: This essay is underdeveloped and difficult to follow, with significant issues in structure and language.
Strengths: The attempt to engage with the topic is visible.
Areas for Improvement: Focus on organizing your thoughts clearly, using proper grammar, and fully developing your main ideas.
Deberta Large Balanced
Predicted Score: 0.007009458728134632
Overall Evaluation: This essay is underdeveloped and difficult to follow, with significant issues in structure and language.
Strengths: The attempt to engage with the topic is visible.
Areas for Improvement: Focus on or